In [ ]:
# !cat /proc/meminfo # Inquire system memory
# !df -h / | awk '{print $4}' # Inquire system disk
# !lscpu  # Inquire system processor

In [ ]:
import numpy as np 
import pandas as pd 
import gc
import ast

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

!pip install --upgrade orjson
import orjson

import os
import sys
def print_log(string):
    os.system(f'echo \"{string}\"')

import dask
import dask.dataframe as dd
from dask.distributed import Client, progress, wait
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()
import numexpr 
numexpr.set_num_threads(1)

## Reading and viewing the dataset

Let's load all the concerned datasets. So the *df_train* here contains main dataset, *df_media* contains all the **title**s with their corresponding duration if they are of **type** *Clip* and ordered in the ideal sequence of the game, and the *df_specs* contains the specifications of all **event_id**s with their information usage and arguments.

In [ ]:
def load_dataset():
    data_dtypes = {"type": "category", "title": "category", "event_data": "str", "event_id": "str", "installation_id": "str", "event_code": "category",
                    "world": "category", "str": "category","event_count": "uint8", "game_time": "uint32"}

    print_log("Reading training data ...")
    df_train = dd.read_csv("../input/data-science-bowl-2019/train.csv", parse_dates=["timestamp"], dtype=data_dtypes)
    
    media_dtypes = {"title": "category","type": "category", "duration": "float64"}
    print_log("Reading media sequence ...")
    df_media = pd.read_csv("../input/data-science-bowl-2019-media-sequence/media_sequence.csv", dtype=media_dtypes)

    print_log("Reading specifications for event ids ...")
    df_specs = pd.read_csv("../input/data-science-bowl-2019/specs.csv")
    
    return df_train, df_media, df_specs

In [ ]:
df_train, df_media, df_specs = load_dataset()
df_media.duration = df_media.duration.fillna(0).astype('float32')
df_titles = df_media.title.unique().tolist()
assessment_attempt_event_code_query = "((event_code=='4100' & title!='Bird Measurer (Assessment)') | (event_code=='4110' & title=='Bird Measurer (Assessment)'))"
attempts_query = "type=='Assessment' & " +  assessment_attempt_event_code_query
print_log("Finised loading datasets")

In [ ]:
df_train.head()

The data doesn't have really that much columns where we can gain additional insights/signals for making our predictions later and is really straightforward as it is. But notice the **event_data** column seems to contain much more data giving details in a specific event of the gameplay.

When we take a peek at one of the specifications for the event ids, we can see that each **event_id** represents a specific detail about the gameplay supported with data provided by the **args** column. We'll take a look at a particular **event_id** namely ***2b9272f4*** in this case to get an idea of what an **info** and **args** value looks like.

In [ ]:
df_specs.head()

In [ ]:
df_specs.query("event_id=='2b9272f4'")["info"].values

In [ ]:
df_specs.query("event_id=='2b9272f4'")["args"].values

Though as we investigate further, even though there are 386 unique event ids, there are only 168 unique values and 191 unique values for the **info** and **args** attributes respectively. This might mean that there are event ids that totally represent the same info, but might represent only a specific **title** (e.g Sandcastle Builder)

In this case we'll focus on the unique **args** for the event ids since we can only based from there what data we can extract from any given **event_id** after all. From now here on, when we say **args**, we could refer it as the actual array of arguments/attributes like the ones shown above or the actual argument/attribute contained in the array (e.g *round*), and we'll use the context interchangeably.

In [ ]:
df_specs.describe()

So let's encode those **args** and map them correspondingly on the **event_id**s of our dataset. After that, we'll then plot a heatmap to see which kind of **args** are represented by the different **event_id**s

> df_specs contains the specifications and info about each available event_id that can be seen in the dataset which is contained in df_train

## Label-encoding each unique args in the df_specs and df_train

In [ ]:
print_log("Encoding args column ...")
args_to_int_map = {args: i for i, args in enumerate(df_specs["args"].unique())}
df_specs["args_encoded"] = df_specs["args"].map(args_to_int_map)

eventid_to_argsint_map = df_specs.set_index("event_id")["args_encoded"].to_dict()
df_train = df_train.assign(**{"args_encoded": df_train["event_id"].map(eventid_to_argsint_map)})
df_train.head()

In [ ]:
def generate_argstype_map(argstype_encoded, use_client=False):
    unique_args = df_specs[argstype_encoded].unique()
    dask_with_args = [df_train[df_train[argstype_encoded]==i].title.unique() for i in range(len(unique_args))]
    if use_client:
        dask_with_args = client.compute(dask_with_args)
        progress(dask_with_args, notebook=False)
        titles_with_args = client.gather(dask_with_args)
    else:
        titles_with_args = dask.compute(dask_with_args)[0]
    titles_with_args_i = {i: titles_with_args[i].tolist() for i in range(len(unique_args))}
    return titles_with_args_i

In [ ]:
def generate_titleargs_table(titles_with_args):
    # Create a dataframe with all the titles and the corresponding args an event id might represent
    # The dataframe is binary, 1 if present, 0 otherwise
    args_length = len(titles_with_args)

    title_args_table = list()
    for title in df_titles:
        title_args_row = [0.0 for i in range(args_length)]
        title_args_table.append(title_args_row)

    args_encoded_col = list(range(args_length))
    title_args_table = pd.DataFrame(title_args_table, 
                                    index=df_titles,
                                    columns=args_encoded_col)\
        .reset_index().rename(columns={"index":"title"}).set_index("title")

    # Change the value to 1 if a given args can be represented by a given title
    for i in range(args_length):
        titles_with_args_i = titles_with_args[i]
        title_args_table.loc[titles_with_args_i, i] = 1

    # Add the type (e.g Game) of each title and group them accordingly
    title_args_table = title_args_table.join(df_media.set_index("title")["type"])\
            .sort_values(["type", "title"]).reset_index()
    title_args_table["title"] = title_args_table["title"].str.cat(title_args_table["type"], sep=" ")
    title_args_table = title_args_table.set_index("title").drop(columns=["type"])
    
    return title_args_table

In [ ]:
def clear_figure(f):
    f.clf()
    plt.clf()
    plt.close(f)
    gc.collect()

In [ ]:
args_encoded_map = generate_argstype_map("args_encoded")
title_args_table = generate_titleargs_table(args_encoded_map)

f = plt.figure(figsize=(20,20))
sns.heatmap(title_args_table, linewidths=.25).set_title("Encoded Args per Title")
f.savefig("encoded_args_vs_title.png")

There are indeed **args** which can represent a majority of the **title**s and take note that there are four types of **title** in the dataset: *Activity*, *Game*, *Clip*, and *Assessment*. You could notice that all *Clips* are only represented by one kind of **args** so we'll disregard it for now so we can see a clearer picture for the rest of the **args**.

In [ ]:
clear_figure(f)
f = plt.figure(figsize=(20,20))
sns.heatmap(title_args_table[~title_args_table.index.str.contains("Clip")], linewidths=.25).set_title("Encoded Args per Title (excluding Clip Titles)")
f.savefig("encoded_args_vs_title2.png")

## Re-encoding args based only on int values they contain

You could notice that some **args**, like the ones on the right most side of the heatmap, are only present in a specific **title**. But remember each unique **args** contains mixed datatypes such as string or an int and we only based their uniqueness based on their *string representation* per se, that's probably why other **args** are so unique that they're only present on a specific title. 

For now, we'll be only examining **args** which has a type *int*, so let's do that and remove non-numeric data in each **args** and re-encode our **args**

In [ ]:
clear_figure(f)
print_log("Extracting and encoding different args type ...")
int_to_args_map = dict()
type_list = ["int", "string", "object", "array"]
for _type in type_list:
    int_to_args_map[_type] = dict()

# Extract numeric attributes of a given args
for i, args in enumerate(df_specs["args"].unique()):
    args_json = orjson.loads(args)
    args_list = dict()
    for _type in type_list:
        args_list[_type] = list()
    
    for args in args_json:
        if (args["name"] != "event_code" and args["name"] != "game_time" and args["name"] != "event_count"):
            # Include numeric types and the "correct" attribute and don't consider those that are already extracted (e.g game_time)
            if (args["type"] == "int" or args["type"] == "number" or args["type"] == "boolean"):  
                args_list["int"].append(args["name"])
            elif ("array" in args["type"]):
                args_list["array"].append(args["name"])
            else:
                args_list[args["type"]].append(args["name"])
    
    for _type in type_list:
        int_to_args_map[_type][i] = args_list[_type]

# Encode corresponding numeric_args
for _type in type_list:
    args_name = "_".join(["args", _type])
    args_encode_name = "".join(["args", _type]) + "_encoded"
    
    args_to_encode_map = {args: int_to_args_map[_type][i] for i, args in enumerate(df_specs["args"].unique())}
    df_specs[args_name] = df_specs["args"].map(args_to_encode_map).astype(str)

    argstype_to_int_map = {args: i for i, args in enumerate(df_specs[args_name].unique())}
    df_specs[args_encode_name] = df_specs[args_name].map(argstype_to_int_map)

    # Set the encoding in the dataset
    eventid_to_args_map = df_specs.set_index("event_id")[args_encode_name].to_dict()
    df_train = df_train.assign(**{args_encode_name: df_train["event_id"].map(eventid_to_args_map)})

df_train.head()

In [ ]:
print_log("Plotting heatmap for encoded args of type int ...")
argsint_encoded_map = generate_argstype_map("argsint_encoded")
title_args_table = generate_titleargs_table(argsint_encoded_map)

# plot heatmap
f = plt.figure(figsize=(20,10))
sns.heatmap(title_args_table[~title_args_table.index.str.contains("Clip")], linewidths=.25).set_title("Int type Encoded Args per Title")
f.savefig("int_encoded_args_vs_title.png")

We could see that we have greatly reduced the uniqueness of our **args**, but also take note that we've loss valuable information from the *string* type attributes of **args** but we'll examine them later on. There's still some **args** that are very specific to a particular **title**, so let's inspect the args of each title available to see more clearly what they have to offer.

We will not be considering *Clips* since it has no any additional info in the event data and we won't also consider *Assessments* since those are the ones that are involved in our prediction. We will need to predict how many attempts a player will make before getting the correct answer regardless of the kind of Assessment (there are 4 types of *Assessments*), so in my opinion it won't make any sense to consider those.

In [ ]:
clear_figure(f)
print_log("Plotting heatmap title per int args ...")
title_allargs = set()
title_args_map = dict()
title_singleargs_map = dict()

for title in title_args_table.itertuples():
    args_list = list()
    args_set = set()
    title = pd.Series(title)
    
    if "Clip" not in title.get(0) and "Assessment" not in title.get(0):
        for i in title_args_table.columns:
            # add +1 since 0 represents the title of the series
            if title.get(i+1) == 1:
                args = ast.literal_eval(df_specs[df_specs.argsint_encoded==i].head(1).args_int.values[0])
                title_allargs.update(args)
                args_set.update(args)
                args_list.append((i, args))
                
        title_args_map[title.get(0)] = args_list
        title_singleargs_map[title.get(0)] = args_set    

We'll then create a heatmap to plot the filtered **args** against each **title** to get a better grasp of the commonalities they have with other titles.

In [ ]:
title_singleargs_table = [[1 if arg in args else 0 for arg in title_allargs] for title, args in title_singleargs_map.items()]
title_singleargs_table = pd.DataFrame(title_singleargs_table, index=title_singleargs_map.keys(), columns=title_allargs)

f = plt.figure(figsize=(20,10))
sns.heatmap(title_singleargs_table, linewidths=.25).set_title("Individual Int Args per Title")
f.savefig("individual_int_args_vs_title.png")

It seems that almost of the title args are somewhat related. They almost all have *round*, *duration*, *level*, etc. but others have a very specific args such as *flower* and alike. We could clearly see that both *Activities* and *Games* have *duration*, *total_duration*, and *dwell_time* in common. While in *Games*, they have *misses*, *correct*, and *round* in common.

## Utilizing the info column in df_specs

But to be sure that we can filter thoroughly and extract useful insights for a given **event_id**, let's utilize the **info** column we had in the event ids specifications. Let's take a rough look at few of them.

In [ ]:
df_specs.sample(10, random_state=1)["info"].values

As we can see above, the **info** are somehow structured in the way what it describes the whole event. It first describes the event itself, then it tells what it contains, then how the event can be utilized including what questions can be answered by them or what the player is currently experiencing. Let's parse each of those info to see in general what type of info these **event_id**s can offer.

In [ ]:
# Looking for phrases that contains "It contains information"
import random
random.sample(set([info[info.find("It contains"):info.find("We")] for info in df_specs["info"].unique().tolist()]), 10)

In [ ]:
# Looking for phrases that contains "players feel are too difficult"
set(info for info in df_specs["info"].unique().tolist() if "difficult" in info)

In [ ]:
# Looking for phrases that contains "We can answer questions like"
set([info[info.find("We can"):] for info in df_specs["info"].unique().tolist()])

There's a potential information we can get for considering the **info** of each event ids that had such statements like **"We can answer questions like how much time elapsed while the game was presenting instruction?"** or **"It helps identify points that players feel are too difficult"**. 

## Classifying event_id by info column

We'll take a look at each info and classify it into four based on the insights we've gathered in the info column; namely, **Diagnosis, AnswerQuestions (we may just refer to it as either Answer or Question), Difficult/Difficulty**, and the rest will be **None**.

In [ ]:
clear_figure(f)
print_log("Classifying event ids based on info column ...")

df_specs["info_str"] = df_specs["info"].map(lambda info: "".join(e for e in info if e.isalnum()))
df_specs["contains_diagnosis"] = df_specs.info_str.str.contains("diagnose")
df_specs["contains_answer"] =  df_specs.info_str.str.contains("answerquestions") 
df_specs["is_difficult"] = df_specs.info_str.str.contains("difficult") 
df_specs["info_type"] = "none"
df_specs.loc[df_specs.contains_diagnosis, "info_type"] = "contains_diagnosis"
df_specs.loc[df_specs.contains_answer, "info_type"] = "contains_answer"
df_specs.loc[df_specs.is_difficult, "info_type"] = "is_difficult"

f, axes = plt.subplots(1, 4, figsize=(20,5))
axes[0].set_title("Event ID count per Info Type")
sns.countplot(y="info_type", data=df_specs, ax=axes[0], order=["contains_answer", "contains_diagnosis", "is_difficult", "none"])
axes[1].set_title("Event ID count per Diagnosis Info Type")
sns.countplot(x="contains_answer", data=df_specs, hue="contains_diagnosis", ax=axes[1])
axes[2].set_title("Event ID count per Difficult Info Type")
sns.countplot(x="contains_diagnosis", data=df_specs, hue="is_difficult", ax=axes[2])
axes[3].set_title("Event ID count per Answer Info Type")
sns.countplot(x="is_difficult", data=df_specs, hue="contains_answer", ax=axes[3])
f.savefig("event_id_count_per_info.png")

The graph clearly indicates that no **event_id** have really the same type and cannot be classified more than one. But mainly, we will be focusing on those **event_id**s who can be used to diagnose the player's gameplay based on their respective stated **info**. 

In [ ]:
clear_figure(f)
def extract_singleargs_per_infotype(args_type, title_args_table):
    reduced_all_args = dict()
    title_singleargs_map = dict()

    args_name = "_".join(["args", args_type])
    args_encode_name = "".join(["args", args_type]) + "_encoded"
    
    info_types = df_specs.info_type.unique().tolist()
    for info_t in info_types:
        reduced_all_args[info_t] = set()

    for title in title_args_table.itertuples():
        args_list = dict()
        args_set = dict()
        for info_t in info_types:
            args_list[info_t] = list()
            args_set[info_t] = set()

        title = pd.Series(title)
        if "Clip" not in title.get(0) and "Assessment" not in title.get(0):
            for i in title_args_table.columns:
                # add +1 since 0 represents the title of the series
                if title.get(i+1) == 1:
                    args_row = df_specs[(df_specs[args_encode_name]==i)].groupby(["info_type"]).head(1)
                    for args_by_type in args_row.itertuples():
                        args = ast.literal_eval(getattr(args_by_type, args_name))
                        info_type = args_by_type.info_type
                        args_set[info_type].update(args)
                        args_list[info_type].append((i, args))
                        reduced_all_args[info_type].update(args)
            title_singleargs_map[title.get(0)] = args_set 
    return title_singleargs_map, reduced_all_args

In [ ]:
import math
def plot_eventids_per_infotype(title_singleargs_map, reduced_all_args, title):
    info_types = df_specs.info_type.unique().tolist()
    row_plot_count = math.ceil(len(info_types))
    f, axes = plt.subplots(row_plot_count, 1, figsize=(20,15*row_plot_count))
    i = j = 0
    
    for info_t in info_types:
        title_singleargs_table = [[1 if arg in args[info_t] else 0 for arg in reduced_all_args[info_t]] for title, args in title_singleargs_map.items()]
        title_singleargs_table = pd.DataFrame(title_singleargs_table, index=title_singleargs_map.keys(), columns=reduced_all_args[info_t])
        
        
#         i = (i+1) if j==2 else i
#         j = 0 if j==2 else j
        axes[i].title.set_text("Event IDs with " + info_t + " info type")
        if len(reduced_all_args[info_t]) == 0:
            title_singleargs_table = pd.DataFrame(np.zeros((len(title_singleargs_map.keys()), 1)), index=title_singleargs_map.keys(), columns=["none"])
        sns.heatmap(title_singleargs_table, linewidths=.25, ax=axes[i])
        i = i + 1
        
    f.savefig(title)

In [ ]:
print_log("Plotting and processing all args type ...")
plot_eventids_per_infotype(*extract_singleargs_per_infotype("int", title_args_table), "individual_args_vs_title_per_info_type.png")

From the *Diagnosis* type heatmap, we can clearly utilize the *round* and *correct* for the *Games*, and *dwell_time* for both *Games* and *Activites* as they're not bound to any specific **title**s though two of them don't have it. While in the *Questions* type heatmap, there's also *round*, *duration*, and *total_duration* that we can use though we must take note that the context/usage of the *round* might be different from the *round* **args** mentioned in the first heatmap as they're totally represent a different set of **event_id**s.

For the *Difficulty* type heatmap, we only have again, a *round* **args**. But maybe, what we can do instead is for each presence of one of those **event_id**s, we'll take note that the player is already finding the game hard to continue.

For the last one, the *None* type heatmap, it seems we can somehow utilize the *round*, *misses*, and *duration* args. We'll take a glimpse of what kind of **info** they might be able to give us so let's have a closer look.

## Discovering a new Info Type from None: Missed

Let's take a look if those **event_id**s that contains misses as there int **args**

In [ ]:
df_specs[df_specs.args_int.str.contains("misses")].head()

In [ ]:
df_specs[df_specs.args_int.str.contains("misses")]["info"].unique()

Clearly, those concerned **event_id**s that contains the *misses* **args** can give us the time spent of a player in a level for speed and accuracy, and the number of levels the player has completed in a *Game*, that's great so we'll consider those and create a new info type that we will named as Missed later on.

## Clarifying context usage of Args per Info Type

Since we've already identified what **args** and type of **event_id**s we'll be using, we'll proceed with clarifying if indeed the context usage of each **args** in a given type of **event_id** is the same (*if they mean what we expected them to mean*), so let's do that.

In [ ]:
def find_definition(args_list, info_type, args_type):
    args_list_info = dict()

    for args in args_list:
        args_with_diagnosis_args = df_specs[(df_specs.info_type==info_type) & (df_specs[args_type].str.contains(args))].groupby(["args_encoded"]).head(1)
        args_list_info[args] = set()

        for _args in args_with_diagnosis_args.itertuples():
            for args_comp in orjson.loads(_args.args):
                if args_comp["name"] == args:
                    args_list_info[args].add(args_comp["info"])

    return args_list_info                

In [ ]:
# Context usage for Diagnosis Info Type of event_id(s)
diagnosis_args = ["round", "correct", "dwell_time"]
find_definition(diagnosis_args, "contains_diagnosis", "args_int")              

In [ ]:
# Context usage for Answer Info Type of event_id(s)
answer_args = ["round", "duration", "total_duration"]
find_definition(answer_args, "contains_answer", "args_int")

In [ ]:
# Context usage for Difficult Info Type of event_id(s)
difficult_args = ["round"]
find_definition(difficult_args, "is_difficult", "args_int")

In [ ]:
# Context usage for None Info Type of event_id(s)
none_args = ["round", "misses", "duration"]
find_definition(none_args, "none", "args_int")

Obviously, for the three types of info namely *Diagnosis*, *Answers* and *Difficult*, the context usage of each **args** were concise and clearly defined. As for the *correct* **args**, it is expected to have a lot of definitions since it really differs per title, but they still refer to the same "correct".

While for the *None* type, only the *misses* **args** is clear and the remaining two **args** are ambiguous, so we may need to separate the concerns for those **event_id**s that contains it.

## Creating the Missed Info Type

In [ ]:
# Created the new Missed Info Type
df_specs["is_missed"] = df_specs.args_int.str.contains("misses")
df_specs.loc[df_specs.args_int.str.contains("misses"), "info_type"] = "is_missed"
missed_args = ["round", "misses", "duration"]
find_definition(missed_args, "is_missed", "args_int")

Now that we've narrowed down the definition of *round* and *duration* for the *misses* args, you can notice that we have now a different definition of the aforementioned **args** in regards to the three types of info evaluated earlier (e.g *Diagnosis*). So to avoid confusion and for future references, we'll mark the corresponding **event_id**s on which type of *round* and *duration* they are representing.

In [ ]:
df_specs["current_round"] = df_specs.args.str.contains("number of the current round")
df_specs["completed_round"] = df_specs.args.str.contains("number of the round that has just been completed")
df_specs["feedback_media_playback_duration"] = df_specs.args.str.contains("media playback in milliseconds")
df_specs["total_fbmedia_playback_duration"] = df_specs.args.str.contains("media playback in milliseconds (if it ran uninterrupted)")
df_specs["level_duration"] = df_specs.args.str.contains("duration of the level in milliseconds")
df_specs["round_duration"] = df_specs.args.str.contains("duration of the round in milliseconds")
df_specs.head()

Here are the new heatmaps for the following **info_type** as we added a new ones named *is_missed*.

In [ ]:
plot_eventids_per_infotype(*extract_singleargs_per_infotype("int", title_args_table), "individual_args_vs_title_per_info_type2.png")

## Examining other types of Args

Now that we're done analyzing those **args** with *int* type, we'll proceed with examining the other types (e.g *string*, *array*) and let's see if we can get more insights in them. What we'll do is like we did for type *int*, plot each *args type* (e.g *string*) and if we find any common **args** for each **title**, we'll find it's definition and decide whether to consider it or not in our analysis later on.

First, we'll go with the *string* type.

In [ ]:
argsstring_encoded_map = generate_argstype_map("argsstring_encoded")
title_args_table = generate_titleargs_table(argsstring_encoded_map)
plot_eventids_per_infotype(*extract_singleargs_per_infotype("string", title_args_table), "individual_stringargs_vs_title_per_info_type.png")

In [ ]:
answer_args = ["identifier", "media_type", "description"]
find_definition(answer_args, "contains_answer", "args_string")

In [ ]:
find_definition(["object"], "contains_diagnosis", "args_string")

In [ ]:
find_definition(["object"], "none", "args_string")

Let's take a quick look on how the **args** *identifier*, *media*, *description*, and *object*  relates to the *int* **args** we've chosen to consider.

In [ ]:
df_specs.loc[df_specs.contains_answer & df_specs.args_string.str.contains("media_type"), df_specs.columns.str.contains("args")].head(1).args.values

In [ ]:
df_specs.loc[df_specs.contains_diagnosis & df_specs.args_string.str.contains("object"), df_specs.columns.str.contains("args")].head(1).args.values

We could see that the **args** *identifier*, *media*, *description* is related to the *duration* in that it describes what kind of media is presented in the duration. While the *object* describes the kind of object that is being dwelled on by the player and *dwell_time* defines its duration.

Let's proceed with the *object* type of args

In [ ]:
argsobject_encoded_map = generate_argstype_map("argsobject_encoded")
title_args_table = generate_titleargs_table(argsobject_encoded_map)
plot_eventids_per_infotype(*extract_singleargs_per_infotype("object", title_args_table), "individual_objectargs_vs_title_per_info_type.png")

In [ ]:
find_definition(["coordinates"], "contains_diagnosis", "args_object")

In [ ]:
find_definition(["coordinates"], "none", "args_object")

In [ ]:
find_definition(["coordinates"], "is_difficult", "args_object")

In [ ]:
df_specs[df_specs.contains_diagnosis & df_specs.args_object.str.contains("coordinates") & df_specs.args_int.str.contains("correct")].head(1).args.values

In [ ]:
df_specs[df_specs.is_difficult & df_specs.args_object.str.contains("coordinates") & df_specs.args_int.str.contains("round")].head(1).args.values

In this case, we won't be considering the *coordinates* **args** as I think it won't really be able to relate this to *correct* **args** for *Diagnosis info type*. Obviously, knowing a coordinate of an answer whether it's correct or not doesn't make sense. For the *Difficult* info type, we might consider the *coordinates* and observe the behavior of a mouse click that may indicate that a player is having difficulties.

But with that in mind, we'll extract all *coordinates* regardless of the *info type* (we'll include the *coordinates* for *Diagnosis* now) to be able to discern mouse clicks that players are having difficulty and players that are not.

Let's proceed with the *array* args type.

In [ ]:
argsarray_encoded_map = generate_argstype_map("argsarray_encoded")
title_args_table = generate_titleargs_table(argsarray_encoded_map)
plot_eventids_per_infotype(*extract_singleargs_per_infotype("array", title_args_table), "individual_arrayargs_vs_title_per_info_type.png")

It seems we can't find any useful **args** for the *array*. As you can see there are also no consistent **args** in the **title**s. So I think we're good and we could start compiling all the **args** and **event_id**s that we'll be extracting in the dataset.

## Filtering the Args to be extracted in the dataset

In [ ]:
print_log("Filtering dataset based on selected event ids ...")
diagnosis_args_int = ["round", "correct", "dwell_time"]
answer_args_int = ["round", "duration", "total_duration"]
missed_args_int = ["round", "misses", "duration"]
difficult_args_int = ["round"]

answer_args_string = ["identifier", "media_type", "description"]

diagnosis_filter = df_specs.contains_diagnosis &\
         (df_specs.args_string.str.contains("object") |\
          df_specs.args_int.str.contains(diagnosis_args_int[1]) | df_specs.args_int.str.contains(diagnosis_args_int[2]))

answer_filter = df_specs.contains_answer &\
            (df_specs.args_int.str.contains(answer_args_int[1]) |\
            df_specs.args_int.str.contains(answer_args_int[2]) | df_specs.args_string.str.contains(answer_args_string[0]) |\
            df_specs.args_string.str.contains(answer_args_string[1]) | df_specs.args_string.str.contains(answer_args_string[2]))

difficult_filter = df_specs.is_difficult & (df_specs.args_object.str.contains("coordinates"))

missed_filter = df_specs.is_missed &\
         (df_specs.args_int.str.contains(missed_args_int[1]) |\
          df_specs.args_int.str.contains(missed_args_int[2]))

df_specs["filtered_id"] = diagnosis_filter | answer_filter | missed_filter
event_id_filter = df_specs[df_specs.filtered_id].event_id.values

df_specs.to_parquet("df_specs.parquet")

We didn't consider the presence of *round* **args** when we build the filter since we expect it to be present for the remaining selected **args**. Also it is too common to all **event_id**s which would expand our **event_id** selection needlessly. We won't also include the args_object such as the *coordinates* and will be reserved for future work and improvement. 

Finally we'll filter out those event ids that are of **type** *Clip* and *Assessment* and use that final list to filter out the whole dataset.

In [ ]:
dask.compute(df_train.shape)

In [ ]:
df_train.head()

In [ ]:
print_log("Removing installation ids without attempts ...")
ids = dask.compute(df_train.query(attempts_query).installation_id.unique())[0]  # ids who took assessments
df_train = df_train[df_train.installation_id.isin(ids.tolist())]
df_train = df_train.assign(**{"index":df_train.installation_id.str.cat([
                    df_train.game_session, 
                    df_train.event_count.astype("str"), 
                    df_train.game_time.astype("str"), 
                    df_train.timestamp.dt.strftime("%y%m%d%H%M%S")])})\
                .set_index("index")

df_event_id_filter = df_train[df_train.event_id.isin(event_id_filter) & (df_train["type"]!="Clip") & (df_train["type"]!="Assessment")]
df_event_id_filter.head()

## Extracting the Args from the filtered dataset

To fully utilize this json extraction process, we would extract all *int* and *string* types of **args** that we could from the **event_id**s we've filtered since it just takes a matter of getting the value of the attribute from the already parsed json.

In [ ]:
all_args_int = set()
all_args_string = set()
# all_args_object = dict()

df_event_id_filter_list = dask.compute(df_event_id_filter.event_id.unique())[0].tolist()
selected_event_ids = df_specs[df_specs.event_id.isin(df_event_id_filter_list)]
for event_id in selected_event_ids.itertuples():
    all_args_int.update(ast.literal_eval(event_id.args_int))
    all_args_string.update(ast.literal_eval(event_id.args_string))

#     all_args.update(ast.literal_eval(event_id.args_object))
#     event_args = orjson.loads(event_id.args)
#     for arg_obj in ast.literal_eval(event_id.args_object):
#         for event_arg in event_args:
#             if event_arg["name"] == arg_obj:
#                 event_arg_items = event_arg["info"][event_arg["info"].find("{") : event_arg["info"].find("}")+1]
#                 event_arg_items = re.findall(r"(?!\")\w*(?=\")", event_arg_items)
#                 if len(event_arg_items) > 0:
#                     all_args_object[event_arg["name"]] = event_arg_items
# import pickle
# with open("all_args.pkl", "wb") as write_args:
#     pickle.dump(all_args, write_args)
# all_args

In [ ]:
len(all_args_int) + len(all_args_string)

We have a total of 36 **args** to extract, we'll then save this as a parquet file so we can use its output to proceed with our data analysis without rerunning the whole notebook.

In [ ]:
filter_columns = [col for col in df_train.columns if "encoded" in col]
df_train_filter = df_train[df_train.event_id.isin(df_event_id_filter_list)]
df_train_filter = df_train_filter.assign(**{args: -1 for args in all_args_int})\
    .assign(**{args: "" for args in all_args_string})\
    .drop(columns=["event_id", "installation_id", "event_code", "title", "type", "world", "game_session", "event_count", "game_time"])\
    .drop(columns=filter_columns)
df_train_filter.head()

In [ ]:
def extract_event(df):
    index_list = df.index.tolist()
    event_object_list = df.event_data.map(lambda x: orjson.loads(x)).tolist()
    event_object_map = dict(zip(index_list, event_object_list))
    for args in all_args_int:
        df[args] = df.index.map(lambda x: event_object_map[x].get(args, -1))
        df[args] = df[args].astype("int") 
    for args in all_args_string:
        df[args] = df.index.map(lambda x: event_object_map[x].get(args, ""))
        df[args] = df[args].astype("category") 
    return df
print("Extracting json args in the dataset ...")
client = Client()
df_train_filter = df_train_filter.map_partitions(extract_event, meta=df_train_filter).drop(columns=["event_data", "timestamp"])
df_train_join = df_train.join(df_train_filter)
dd_train_future = client.persist(df_train_join)
progress(dd_train_future, notebook=False)   
dd_train_future.to_parquet("df_train.parquet")